In [20]:
import pandas as pd
import numpy as np 
import itertools
import keras
from keras.applications.inception_v3 import InceptionV3
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, LeakyReLU 
from keras import applications  
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras import backend as K


import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math  
import datetime
import time

In [2]:
#Set default dimensions for model
img_width, img_height = 299, 299  
   
#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model-inceptionV3.h5' 

#path for datasets
train_data_dir = 'storage/train'  
validation_data_dir = 'storage/valid'
test_data_dir = 'storage/test'
   
# number of epochs to train top model  
epochs = 50

# batch size used by flow_from_directory and predict_generator  
batch_size = 50

In [25]:
inceptionV3 = InceptionV3(include_top=False, weights='imagenet')

In [10]:
datagen = ImageDataGenerator(rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)  

In [11]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

start = datetime.datetime.now()

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_train_samples = len(generator.filenames)
num_classes = len(generator.class_indices)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))

bottleneck_features_train = inceptionV3.predict_generator(generator, predict_size_train)

np.save('bottleneck_features_train-inceptionV3.npy', bottleneck_features_train)

end= datetime.datetime.now() 
elapsed= end-start 
print ('Time: ', elapsed)

Found 3867 images belonging to 6 classes.
Time:  0:08:33.461106


In [12]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

start = datetime.datetime.now() 

datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_validation_samples = len(generator.filenames)

predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

bottleneck_features_validation = inceptionV3.predict_generator(
generator, predict_size_validation)

np.save('bottleneck_features_validation-inceptionV3.npy', bottleneck_features_validation)

end= datetime.datetime.now() 
elapsed= end-start 
print ('Time: ', elapsed)

Found 1030 images belonging to 6 classes.
Time:  0:02:29.287820


In [13]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

start = datetime.datetime.now()

datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_test_samples = len(generator.filenames)

predict_size_test = int(math.ceil(nb_test_samples / batch_size))

bottleneck_features_test = inceptionV3.predict_generator(
generator, predict_size_test)

np.save('bottleneck_features_test-inceptionV3.npy', bottleneck_features_test) 

end= datetime.datetime.now() 
elapsed= end-start 
print ('Time: ', elapsed)

Found 1018 images belonging to 6 classes.


/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33554432 bytes but only got 0. Skipping tag 516
  " Skipping tag %s" % (size, len(data), tag))
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 69337600 bytes but only got 0. Skipping tag 3072
  " Skipping tag %s" % (size, len(data), tag))
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19988480 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2046820352 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag

Time:  0:02:09.067830


In [3]:
datagen_top = ImageDataGenerator(rescale=1./255)

In [4]:
#training data
generator_top = datagen_top.flow_from_directory(  
         train_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
   
nb_train_samples = len(generator_top.filenames)  
num_classes = len(generator_top.class_indices)  
   
# load the bottleneck features saved earlier  
train_data = np.load('bottleneck_features_train-inceptionV3.npy')  
   
# get the class lebels for the training data, in the original order  
train_labels = generator_top.classes  
   
# convert the training labels to categorical vectors  
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 3867 images belonging to 6 classes.


In [5]:
#validation data
generator_top = datagen_top.flow_from_directory(  
         validation_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
   
nb_validation_samples = len(generator_top.filenames)  
   
validation_data = np.load('bottleneck_features_validation-inceptionV3.npy')  
   

validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

Found 1030 images belonging to 6 classes.


In [6]:
#testing data
generator_top = datagen_top.flow_from_directory(  
         test_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
   
nb_test_samples = len(generator_top.filenames)  
   
test_data = np.load('bottleneck_features_test-inceptionV3.npy')  
   

test_labels = generator_top.classes  
test_labels = to_categorical(test_labels, num_classes=num_classes)

Found 1018 images belonging to 6 classes.


In [26]:
NAME = 'Inception V3 CNN-{}'.format(int(time.time()))


#This is the best model we found. For additional models, check out I_notebook.ipynb
start = datetime.datetime.now()
model = Sequential()

model.add(Flatten(input_shape=train_data.shape[1:]))

model.add(Dense(256, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))  
model.add(Dense(64, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])  

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=5)
checkpoint = ModelCheckpoint(filepath='inceptionV3.h5', monitor='val_acc', save_best_only=True)
early_stop = EarlyStopping(monitor='acc', patience=2)

history = model.fit(train_data, train_labels,
                    epochs=70,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    validation_split=0.2,
                    callbacks=[tensorboard, reduce_lr, checkpoint, early_stop])  

model.save_weights(top_model_weights_path)  

(eval_loss, eval_accuracy) = model.evaluate(  
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))  
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Train on 3867 samples, validate on 1030 samples
Epoch 1/70
3867/3867 [==============================] - 18s 5ms/step - loss: 4.2995 - acc: 0.3452 - val_loss: 0.8101 - val_acc: 0.7204
Epoch 2/70
3867/3867 [==============================] - 17s 4ms/step - loss: 2.7202 - acc: 0.5045 - val_loss: 0.6746 - val_acc: 0.7932
Epoch 3/70
3867/3867 [==============================] - 17s 4ms/step - loss: 2.4946 - acc: 0.5759 - val_loss: 0.7826 - val_acc: 0.8039
Epoch 4/70
3867/3867 [==============================] - 17s 4ms/step - loss: 2.1724 - acc: 0.6281 - val_loss: 0.8864 - val_acc: 0.7854
Epoch 5/70
3867/3867 [==============================] - 18s 5ms/step - loss: 2.0803 - acc: 0.6527 - val_loss: 0.6916 - val_acc: 0.8398
Epoch 6/70
3867/3867 [==============================] - 16s 4ms/step - loss: 1.9561 - acc: 0.6768 - val_loss: 1.3621 - val_acc: 0.7417
Epoch 7/70
3867/3867 [==============================] - 17s 4ms/step - loss: 1.7323 - acc: 0.7179 - val_loss: 1.0344 - val_acc: 0.7971
Epoch 8

In [ ]:
model.summary()

In [ ]:

#Graphing our training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('InceptionV3 Training and validation accuracy')
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('InceptionV3 Training and validation loss')
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend()
plt.show()